In [255]:
#import dependecies

import pandas as pd


In [267]:
#import file data

gamers_file = "../Resources/purchase_data.csv"
gamers_data = pd.read_csv(gamers_file)
gamers_data

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,92,Final Critic,4.19


In [268]:
#count how many players

players_count = len(gamers_data["SN"].unique())
players_count_table = pd.DataFrame({"Total Players":[players_count]})
players_count_table

,Total Players
0,576


In [269]:
#Purchasing Analysis (Total) - Number of Unique Items - Average Price - Number of Purchases - Total Revenue

unique_items = len(gamers_data["Item Name"].unique())
average_price = round(gamers_data["Price"].mean(),2)
number_of_purchases = gamers_data["Item ID"].count()
total_revenue = round(sum(gamers_data["Price"]),2)

#Create dataframe for purchasing analysis

formater = {'Average Price':"${:.2f}",'Total Revenue':"${:,.2f}"}
purchasing_analysis = pd.DataFrame({"Number of Unique Items":[unique_items],"Average Price": [average_price],
                                    "Number of Purchase":[number_of_purchases],"Total Revenue": [total_revenue]})

purchasing_analysis.style.format(formater)

,Number of Unique Items,Average Price,Number of Purchase,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [270]:
#Gender Demographic

#Group by Gender, grab uniques
gender_data = gamers_data.groupby('Gender').agg({'SN': "nunique"})

#Setup % column
perc_data = round(gender_data["SN"] / 576 *100,2).astype(str) + '%'

#Add % column
merged_gender_data = pd.merge(gender_data, perc_data, on='Gender', how='outer')

#Change names
merged_gender_data.columns = ['Total Count','Percentage of Players']

#Set % from high to low
sorted_gender_data = merged_gender_data.sort_values('Percentage of Players', ascending= False)
 
sorted_gender_data




,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [271]:
# Purchasing Analysis (Gender)

gender_purch = gamers_data.groupby('Gender').agg({"Item ID": "count","Price": ["mean", "sum"]}).round(2)

gender_purch.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value"]

total_purch = round(gender_purch['Total Purchase Value'] / sorted_gender_data['Total Count'],2)

merged_gender_purch = pd.concat([gender_purch, total_purch.rename("Avg Total Purchase per Person")], axis=1)

formater_2 = {'Average Purchase Price':"${:.2f}",'Total Purchase Value':"${:,.2f}",'Avg Total Purchase per Person':"${:.2f}"}

merged_gender_purch.style.format(formater_2)



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [272]:
#Age Demographics

#bins and group setup
bins = [0,9.9,14,19.99,24,29.99,34,39.99,100]

age_groups = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

pd.cut(gamers_data["Age"], bins, labels=age_groups)

gamers_data['Age Bucket'] = pd.cut(gamers_data["Age"], bins, labels=age_groups)

#group columns, set ages with unique SNs

age_demo = gamers_data.groupby('Age Bucket').agg({"SN":"nunique"})

per_o_players = round(age_demo["SN"] / 576 * 100,2).astype(str) + '%'

#merge grouby with percentage column

merged_age_demo = pd.merge(age_demo, per_o_players, on = "Age Bucket",  how ="outer")

merged_age_demo.columns = ['Total Count','Percentage of Players']

merged_age_demo

#age_dem_perc = pd.cut(gamers_data["Age"], bins, labels=age_groups)

#age_dem_perc.describe()


,Total Count,Percentage of Players
Age Bucket,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [273]:
# Purchasing Analysis (Age)

#group first three columns

age_demo = gamers_data.groupby('Age Bucket').agg({"SN": "count","Price": ["mean","sum"]})

age_demo.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value"]

avg_total_purch = age_demo['Total Purchase Value'] / merged_age_demo['Total Count']

# Concat all columns

merged_purch_analysis = pd.concat([age_demo, avg_total_purch.rename("Avg Total Purchase per Person")], axis=1)

#format $

formater_3 = {'Average Purchase Price':"${:.2f}",'Total Purchase Value':"${:,.2f}", "Avg Total Purchase per Person":"${:.2f}" }

merged_purch_analysis.style.format(formater_3)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Bucket,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [274]:
#Top Spenders

top_spend = gamers_data.groupby('SN').agg({'Item ID': "count", 'Price':["mean","sum"]})

top_spend.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value"]

top_spenders = top_spend.sort_values('Total Purchase Value', ascending= False)

formater_4 = {'Average Purchase Price':"${:.2f}", "Total Purchase Value":"${:.2f}" }

top_spenders.head().style.format(formater_4)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [275]:
#Most Popular Items

most_popular_items = gamers_data.groupby(['Item ID','Item Name']).agg({"Price": ["count","max","sum"]})

most_popular_items.columns = ["Purchase Count","Item Price","Total Purchase Value"]

most_popular = most_popular_items.sort_values('Purchase Count', ascending= False)

formater_5 = {'Item Price':"${:.2f}", "Total Purchase Value":"${:.2f}" }

most_popular.head().style.format(formater_5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.33,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [276]:
#Most Profitable - Filter by Purchase Value

most_profitable = most_popular.sort_values('Total Purchase Value', ascending= False)

most_profitable.head().style.format(formater_5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


In [277]:
# 